# Backpropagation Practice

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [55]:
##### Your Code Here #####
# I want activations that correspond to negative weights to be lower
# and activations that correspond to positive weights to be higher
class NeuralNetwork():
    def __init__(self, input_size = 3, hiddenNodes = 4, outputNodes = 1):
#     def __init__(self, input_size, hiddenNodes, outputNodes):
        # Set up Architecture of neural network
        self.input = input_size 
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes
        
        # Initial Weights
        # 2x3 matrix for first layer
        self.weights1 = np.random.randn(self.input, self.hiddenNodes)
        #3x1 matrix for hidden to output layer
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forawrd

        """

        #weighted sum of inputs and hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        #add bias
#         self.hidden_sum + bias

        #Activations of weighted sums
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        

        #Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)

        # Final Activation of output
        self.activated_output = self.sigmoid(self.output_sum)


        return self.activated_output
    
    def get_attributes(self):
        
        attributes = ['weights1', 'hidden_sum', 'activated_hidden', 'weights2', 'output']

        [print(i + '\n', getattr(nn,i), '\n' + '---'*3) for i in dir(nn) if i in attributes]

    def backward(self, X, y, o):
        """
        Backward propagate through the network
        """
        self.o_error = y - o #error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # apply derivative of sigmoid to error

        self.z2_error = self.o_delta.dot(self.weights2.T) # z2 error: how much our hidden layer weights were off
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden)

        self.weights2 += self.activated_hidden.T.dot(self.o_delta) #adjust second set (hidden => output) weights
        self.weights1 += X.T.dot(self.z2_delta) #Adjust first set (input => hidden) weights

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [45]:
import numpy as np

inputs = np.array(([0,0,1],
                   [0,1,1],
                   [1,0,1],
                   [0,1,0],
                   [1,0,0],
                   [1,1,1],
                   [0,0,0]), dtype=int)

correct_outputs = np.array(([0], [1], [1], [1], [1], [0], [0]), dtype=int)




In [46]:
nn = NeuralNetwork(input_size = 3, hiddenNodes = 4, outputNodes = 1)

X = inputs
y = correct_outputs

for i in range(1000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 50 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X, y)

+---------EPOCH 1---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.32199498]
 [0.36237912]
 [0.32422673]
 [0.41135672]
 [0.44551237]
 [0.35641119]
 [0.44335184]]
Loss: 
 0.27777969847194106
+---------EPOCH 2---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.40465438]
 [0.44405169]
 [0.41321101]
 [0.47383568]
 [0.52545253]
 [0.44588653]
 [0.51131133]]
Loss: 
 0.2542061831760351
+---------EPOCH 3---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.45532002]
 [0.49357677]
 [0.46863913]
 [0.51084434]
 [0.57212803]
 [0.50107213]
 [0.5499022 ]]
Loss: 
 0.24599122549589778
+---------EPOCH 4---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [63]:
##### Your Code Here #####
from sklearn.datasets import load_digits
digits = load_digits()

In [69]:
X = digits.data
X.shape

(1797, 64)

In [65]:
import pandas as pd
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0
5,0.0,0.0,12.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,9.0,16.0,16.0,10.0,0.0,0.0
6,0.0,0.0,0.0,12.0,13.0,0.0,0.0,0.0,0.0,0.0,...,8.0,0.0,0.0,0.0,1.0,9.0,15.0,11.0,3.0,0.0
7,0.0,0.0,7.0,8.0,13.0,16.0,15.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,13.0,5.0,0.0,0.0,0.0,0.0
8,0.0,0.0,9.0,14.0,8.0,1.0,0.0,0.0,0.0,0.0,...,8.0,0.0,0.0,0.0,11.0,16.0,15.0,11.0,1.0,0.0
9,0.0,0.0,11.0,12.0,0.0,0.0,0.0,0.0,0.0,2.0,...,4.0,0.0,0.0,0.0,9.0,12.0,13.0,3.0,0.0,0.0


In [66]:
y = (digits.target).reshape(-1,1)
y.shape

(1797, 1)

In [76]:
new_nn = NeuralNetwork(input_size = 64, hiddenNodes = 65, outputNodes = 1)


# try normalizing X
row_sums = X.sum(axis=1)
X_norm = X / row_sums[:, np.newaxis]
X = X_norm
y = y /100


for i in range(1000):
    if (i+1 in [1,2,3,4,5]) or ((i+1) % 50 ==0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(new_nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - new_nn.feed_forward(X)))))
    new_nn.train(X, y)

+---------EPOCH 1---------+
Input: 
 [[0.         0.         0.0170068  ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.03194888 0.         0.        ]
 [0.         0.         0.         ... 0.04651163 0.02616279 0.        ]
 ...
 [0.         0.         0.0026738  ... 0.01604278 0.         0.        ]
 [0.         0.         0.00581395 ... 0.03488372 0.         0.        ]
 [0.         0.         0.0255102  ... 0.03061224 0.00255102 0.        ]]
Actual Output: 
 [[0.  ]
 [0.01]
 [0.02]
 ...
 [0.08]
 [0.09]
 [0.08]]
Predicted Output: 
 [[0.63565433]
 [0.61840665]
 [0.66947497]
 ...
 [0.66460713]
 [0.64913293]
 [0.72381525]]
Loss: 
 0.36289510129225094
+---------EPOCH 2---------+
Input: 
 [[0.         0.         0.0170068  ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.03194888 0.         0.        ]
 [0.         0.         0.         ... 0.04651163 0.02616279 0.        ]
 ...
 [0.         0.         0.0026738  ... 0.01604278

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?